# Librairies

In [1]:
import portfolio_functions as pf
import numpy as np
import pandas as pd

# Portfolio

Importons les données :

In [2]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
datas = pd.read_csv('datas/prices.csv', sep=',')

# On récupère les tickers stockés dans le fichier tickers.txt
tickers = pf.get_tickers('datas/tickers.txt')

In [3]:
# Uniquement les prix liées aux tickers
prices = datas[["Date"]+tickers]

# On transforme la colonne Date en index
prices.set_index("Date", inplace=True)
prices.index = pd.to_datetime(prices.index)

prices.head()

,AALB.AS,ACOMO.AS,AD.AS,AGN.AS,AJAX.AS,AKZA.AS,ALX.AS,AMG.AS,AMUND.AS,ARCAD.AS,ASM.AS,ASML.AS,AXS.AS,BAMNB.AS,BBED.AS,BESI.AS,BGHL.AS,BRILL.AS,BRNL.AS,CRBN.AS
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,8.183498,3.089914,6.128687,2.645846,6.127661,29.324984,78.000000,8.194963,12.383609,10.344408,11.537248,27.659260,1.967913,4.795937,10.836442,0.830604,9.75,5.382684,8.118333,12.707550
2010-01-05,8.148955,3.078680,6.077343,2.639794,6.165899,29.102234,75.000000,8.169353,12.383609,10.731913,11.630810,27.710567,1.889196,4.772008,11.138389,0.846276,9.90,5.408440,8.564506,12.619114
2010-01-06,8.310164,3.089914,6.111786,2.630441,6.165899,29.177525,72.800003,8.080161,12.383609,10.791023,11.788899,28.007000,1.810480,4.731705,11.084709,0.830604,9.81,5.408440,8.889911,12.772176
2010-01-07,8.267944,3.075870,5.977906,2.683808,5.974709,28.703789,73.000000,8.124315,12.383609,10.804158,11.511437,27.493944,1.771121,4.768229,10.769344,0.836873,9.95,5.408440,8.772497,12.925238
2010-01-08,8.283298,3.101151,6.039648,2.766887,6.070304,28.499855,74.000000,8.106654,12.383609,10.731913,11.927629,26.570450,1.810480,4.961554,10.722376,0.852545,10.13,5.408440,9.557491,13.340207


Calculons les returns journaliers :

In [4]:
returns = prices.pct_change().dropna()

returns.head()

,AALB.AS,ACOMO.AS,AD.AS,AGN.AS,AJAX.AS,AKZA.AS,ALX.AS,AMG.AS,AMUND.AS,ARCAD.AS,ASM.AS,ASML.AS,AXS.AS,BAMNB.AS,BBED.AS,BESI.AS,BGHL.AS,BRILL.AS,BRNL.AS,CRBN.AS
Date,,,,,,,,,,,,,,,,,,,,
2010-01-05,-0.004221,-0.003636,-0.008378,-0.002287,0.006240,-0.007596,-0.038462,-0.003125,0.0,0.037460,0.008110,0.001855,-0.040000,-0.004989,0.027864,0.018868,0.015385,0.004785,0.054959,-0.006959
2010-01-06,0.019783,0.003649,0.005668,-0.003543,0.000000,0.002587,-0.029333,-0.010918,0.0,0.005508,0.013592,0.010697,-0.041666,-0.008446,-0.004819,-0.018518,-0.009091,0.000000,0.037995,0.012129
2010-01-07,-0.005081,-0.004545,-0.021905,0.020288,-0.031008,-0.016236,0.002747,0.005465,0.0,0.001217,-0.023536,-0.018319,-0.021740,0.007719,-0.028450,0.007547,0.014271,0.000000,-0.013208,0.011984
2010-01-08,0.001857,0.008219,0.010328,0.030955,0.016000,-0.007105,0.013699,-0.002174,0.0,-0.006687,0.036155,-0.033589,0.022223,0.040544,-0.004361,0.018726,0.018090,0.000000,0.089484,0.032105
2010-01-11,0.014829,0.012682,0.001076,-0.015510,-0.012441,-0.023338,-0.004054,0.020697,0.0,-0.022338,-0.029213,-0.037546,0.021739,-0.012565,0.001252,-0.003676,0.000000,-0.023809,0.013338,-0.000765


Nous devons définir une période d'entrainement et de test :
* la période d'entrainement est entre la première date disponible et le 1er Janvier 2015
* la période de test est entre la fin de l'entrainement et la dernière date disponible 

In [5]:
begin_train = returns.index[0]
end_train = "2017-01-01"
begin_test = end_train
end_test = returns.index[-1]

Il est nécessaire de séparer les données entre la phase d'entrainement et la phase de test.

In [6]:
returns_train, returns_test = pf.seperate_df(returns, begin_train, end_train, begin_test, end_test)

Nous pouvons déterminer les returns pour chaque année grâce à la ligne de code suivante :

In [8]:
annual_returns_train = returns_train.resample('Y').sum()
annual_returns_test = returns_test.resample('Y').sum()

In [9]:
def annu_rend(portfolio:pd.Series,period_per_year:int=252,show:bool=False):
    """
    Calculate the annualized return of the series considered
    Parameters
    ----------
    portfolio : pd.Series
        Series of the returns considered.
    period_per_year : int
        Period considered to calculate the annualized return.
    show : bool
        To show or not the result.
    Returns
    -------
    float
        Annualized return of the series considered.
    """

    nb_days = portfolio.shape[0]
    final_return = (portfolio+1).prod()
    annu_return = final_return**(period_per_year/nb_days) - 1
    if show:
        print(f'Annualized return on the period: {round(annu_return*100,3)}%.')
    
    return annu_return

def annu_rend_df(df:pd.DataFrame,period_per_year:int=252,arr:bool=False,show:bool=False):
    """
    Calculated annualized return for each asset considered in the DataFrame
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame composed of the returns of the assets.
    period_per_year : int
        Period considered to calculate the annualized return.
    arr : bool
        If false the function returns a dictionary otherwise an array.
    show : bool
        To show or not the result of each annualized return.
    Returns
    -------
    dict or array
        Dictionary of annualized returns of the assets or array of annualized returns of the assets.
    """

    rendements = {}
    for i in df.columns:
        rendements[i] = round(annu_rend(df[i]),5)
    
    if not arr:
        return rendements
    
    else:
        return np.array(list(rendements.values()))

In [13]:
annualized_returns_train  = returns_train.mean()*252
annualized_returns_train

AALB.AS     0.201686
ACOMO.AS    0.256831
AD.AS       0.153714
AGN.AS      0.108382
AJAX.AS     0.099191
AKZA.AS     0.095809
ALX.AS      0.102051
AMG.AS      0.138890
AMUND.AS    1.594121
ARCAD.AS    0.058198
ASM.AS      0.208288
ASML.AS     0.224177
AXS.AS      0.094678
BAMNB.AS    0.071126
BBED.AS     0.084164
BESI.AS     0.441782
BGHL.AS     0.179463
BRILL.AS    0.224388
BRNL.AS     0.116048
CRBN.AS     0.125254
dtype: float64

In [11]:
annu_rend_df(returns_train,show=True)

{'AALB.AS': 0.17914,
 'ACOMO.AS': 0.26442,
 'AD.AS': 0.14463,
 'AGN.AS': 0.04537,
 'AJAX.AS': 0.04573,
 'AKZA.AS': 0.06266,
 'ALX.AS': -0.47773,
 'AMG.AS': 0.07239,
 'AMUND.AS': -0.23452,
 'ARCAD.AS': 0.00877,
 'ASM.AS': 0.17123,
 'ASML.AS': 0.19687,
 'AXS.AS': -0.12279,
 'BAMNB.AS': -0.02612,
 'BBED.AS': 0.05174,
 'BESI.AS': 0.45394,
 'BGHL.AS': 0.08821,
 'BRILL.AS': 0.20831,
 'BRNL.AS': 0.06556,
 'CRBN.AS': 0.08571}